# Trabalho Pŕatico 1 - Computação Gráfica
Feito por Gustavo Leão e Luigi Eliabe

In [201]:
class Window:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [202]:
class Viewport:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [203]:
class Ponto:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
        pass

    def __str__(self) -> str:
        return (f"({self.x},{self.y})")


In [204]:
class Reta:
    def __init__(self,ponto1, ponto2) -> None:
        self.ponto1 = ponto1
        self.ponto2 = ponto2
        pass

    def __str__(self) -> str:
        return (f"({self.ponto1},{self.ponto2})")

In [205]:
class Poligono:

    def __init__(self,*pontos) -> None:
        self.pontosPoligono = []
        for ponto in pontos:
            self.pontosPoligono.append(ponto)
        pass

    
    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosPoligono)+')'

In [249]:
class LerXML:
    import xml.etree.ElementTree as Et

    tree = Et.parse('entrada.xml') #Gera uma árvore com o arquivo
    raiz = tree.getroot()          #Define a raiz


    #Percorre as raizes do arquivos, procurando todas as TAGS com esse nome
    #Nesse caso estamos percorrendo para pegar os valores da ViewPort
    for view in raiz.findall('viewport/vpmin'):
        vpmin = view.attrib #Atribui os atributos da TAG

    for view in raiz.findall('viewport/vpmax'):
        vpmax = view.attrib

    vpminX = float(vpmin.get('x')) #Pega o atributo X
    vpminY = float(vpmin.get('y')) #Pega o atributo Y
    vpmaxX = float(vpmax.get('x'))
    vpmaxY = float(vpmax.get('y'))


    #Vamos pegar agora os valores da Window
    for win in raiz.findall('window/wmin'):
        wmin = win.attrib

    for win in raiz.findall('window/wmax'):
        wmax = win.attrib

    wminX = float(wmin.get('x'))
    wminY = float(wmin.get('y'))
    wmaxX = float(wmax.get('x'))
    wmaxY = float(wmax.get('y'))


    janela = Window(wminX,wminY,wmaxX,wmaxY)            #Criando a instância da Window
    viewport = Viewport(vpminX,vpminY,vpmaxX,vpmaxY)    #Criando a instância da Viewport



    #Pegando os valores dos pontos
    pontos = [] #Criando um vetor de Pontos
    for pon in raiz.findall('ponto'):
        ponto = pon.attrib
        ponX = float(ponto.get('x'))
        ponY = float(ponto.get('y'))
        pontos.append(Ponto(ponX,ponY))                 #Adicionando os pontos ao valores

    for x in pontos:
        print(x)
    #Pegando valores das Retas
    pontoRetas = [] #Criamos um vetor de pontos da reta
    retas = []      #Criamos um vetor com as retas
    i=0             #contador de pontos da reta
    for ret in raiz.findall('reta'):
        for ponto in ret:
            x = float(ponto.attrib.get('x'))
            y = float(ponto.attrib.get('y'))
            pontoRetas.append(Ponto(x,y)) #salvando os pontos da reta em um vetor
        retas.append(Reta(pontoRetas[i],pontoRetas[i+1])) #salvadno a reta em um vetor
        i+=2    
    
    # contador=0 
    # retas = [] #Criamos um vetor de retas
    # while contador < i:
    #     #É adicionado dois pontos a reta
    #     retas.append(Reta(pontoRetas[contador],pontoRetas[contador+1]))
    #     contador+=2     #contador soma +2 pois uma reta sempre é formato de 2 pontos

    #Pegar os pontos do Poligono
    pontosPoligono = [] #Fazemos o mesmo esquema das retas
    poligonos = []
    for poli in raiz.findall('poligono'):
        pontosPoligono.clear()
        for ponto in poli:
            x = float(ponto.attrib.get('x'))
            y = float(ponto.attrib.get('y'))
            pontosPoligono.append(Ponto(x,y))
        poligono = Poligono(*pontosPoligono)
        poligonos.append(poligono) #Mandamos um vetor de pontos para salvar o poli
    
    # poligonos.append(Poligono(pontosPoligono[0],pontosPoligono[1],pontosPoligono[2],pontosPoligono[3]))

(0.0,2.0)
(2.0,4.0)
(4.0,6.0)


In [300]:
class Transformada:

    def __init__(self,win,view) -> None:
        self.win = win
        self.view = view
        pass

    def transformarPonto(self,x,y) -> Ponto:
        self.pontoX = (((x-self.win.minX)/(self.win.maxX-self.win.minX))*(self.view.maxX-self.view.minX))
        self.pontoY = ((1-(y-self.win.minY)/(self.win.maxY-self.win.minY))*(self.view.maxY-self.view.minY))
        return Ponto(self.pontoX,self.pontoY)
    
    def transformarReta(self,ponto1,ponto2) -> Reta:
        self.ponto1 = self.transformarPonto(ponto1.x,ponto1.y)
        self.ponto2 = self.transformarPonto(ponto2.x,ponto2.y)
        return Reta(self.ponto1,self.ponto2)
    
    def transformarPoli(self,pontos) -> Poligono:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Poligono(*pontosTrans)
            
    def __str__(self) -> str:
        return (f"Viewport: {self.view}\nWindow: {self.win}")

In [301]:
if __name__ == '__main__':

    file = LerXML

    trans = Transformada(file.janela,file.viewport)
    print(trans)

    for x in file.pontos:
        print(trans.transformarPonto(x.x,x.y))

    for retas in file.retas:
        print(trans.transformarReta(retas.ponto1,retas.ponto2))

    for poli in file.poligonos:
        print(trans.transformarPoli(poli.pontosPoligono))
   

Viewport: (10.0,10.0,630.0,470.0)
Window: (0.0,0.0,10.0,10.0)
(0.0,368.0)
(124.0,276.0)
(248.0,184.0)
((124.0,368.0),(248.0,276.0))
((248.0,276.0),(124.0,184.0))
((62.0,414.0),(62.0,368.0),(124.0,368.0),(124.0,414.0))
